# Martingale Backtesting 

This notebook will look to generate some sort os trading signals and store it in pandas dataframe column. that waill then be used in backtesting.py to backtest using a martingale risk management approach

In [25]:
import MetaTrader5 as mt5
import pandas as pd
import ta
import time
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import cycler
%matplotlib inline

In [26]:
# Settings 
symbol = 'XAUUSD'
timeframe = mt5.TIMEFRAME_M5
start_date = datetime(2024, 1, 1)
end_date = datetime.now()

# Function 
def GetMTData():
    mt5.initialize()
    bars = pd.DataFrame(mt5.copy_rates_range(symbol, timeframe, start_date, end_date))
    bars['time'] = pd.to_datetime(bars['time'], unit='s')
    bars.columns = bars.columns.str.capitalize()
    return bars

df  = GetMTData()

In [27]:
# Function to generate signals based on EMA crossover
def generate_signals(data):
    # Calculate EMAs
    data['FEMA'] = ta.trend.ema_indicator(data['Close'], window=20)
    data['SEMA'] = ta.trend.ema_indicator(data['Close'], window=120)

    
    data['Signals'] = 0
    
    prev_signal = 0

    # Generate signals based on EMA crossover
    for i in range (1, len(data)):
        if data['FEMA'][i]>data['SEMA'][i] and data['FEMA'][i-1] <= data['SEMA'][i-1]: # Creating buy position == 2 
                data.at[i, 'Signals'] = 2
                prev_signal = 2
        #elif data['FEMA'][i]<data['SEMA'][i] and data['FEMA'][i-1] >= data['SEMA'][i-1]: # Creating sell position == 1 
                #data.at[i, 'Signals'] = 1
                #prev_signal = 1
        else:
            data.at[i, 'Signals'] = 0
            prev_signal = 0
    return data


df = generate_signals(df)

In [28]:
def SIGNAL():
    return df.Signals

In [29]:


class SMAcross(Strategy):
    
    initsize = 50.0
    mysize = initsize
    
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        
        
    def next(self):
        super().next()
        
        # Martingale Risk Management
        if (self.signal1 > 0 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0) :
            self.mysize = round(self.mysize*1.2,2)
        elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
            self.mysize = self.initsize
        
        
        if self.signal1 == 2:
            sl1 = self.data.Close[-1] - 250e-1
            tp1 = self.data.Close[-1] + 550e-1
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1 == 1:
            sl1 = self.data.Close[-1] + 250e-1
            tp1 = self.data.Close[-1] - 550e-1
            self.sell(sl=sl1, tp=tp1, size=self.mysize)   
            

        

In [30]:
bt = Backtest(df, SMAcross, cash=100000, commission= 0.00, exclusive_orders=True, margin=1/100)
            
stat = bt.run()
stat


C:\Users\User\AppData\Local\Temp\ipykernel_14552\290011193.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, SMAcross, cash=100000, commission= 0.00, exclusive_orders=True, margin=1/100)


Start                                     0.0
End                                   18299.0
Duration                              18299.0
Exposure Time [%]                   85.535519
Equity Final [$]                     111134.7
Equity Peak [$]                      111511.2
Return [%]                            11.1347
Buy & Hold Return [%]               11.102611
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                    -4.04752
Avg. Drawdown [%]                   -0.263648
Max. Drawdown Duration                11648.0
Avg. Drawdown Duration             181.064516
# Trades                                108.0
Win Rate [%]                         54.62963
Best Trade [%]                       2.687134
Worst Trade [%]                     -1.238045
Avg. Trade [%]                    

In [31]:
bt.plot()

Row(id='3449', ...)